In [1]:
import lxml.html as html
import re
import time
import io
import nltk
import math
import string
from collections import defaultdict
import urllib2
from bs4 import BeautifulSoup, SoupStrainer
from cookielib import CookieJar

In [2]:
# Cookie enable
cj = CookieJar()
opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))

def getPageText( pagelink ):
        text = []
        page = opener.open('http://izvestia.ru' + pagelink).read()
        #print 'http://izvestia.ru' + pagelink
        response = BeautifulSoup( page , "html.parser", parse_only=SoupStrainer("div", {'class':'text_block'}) )
        for a in response:
            text.append(a.text)
        return '\n'.join(text)

def grabNewForGroup( group, pageLimit ):
    sports_pages = []
    for pageNumber in range(1,pageLimit+1):
        url = '''http://izvestia.ru/archive/{}?type=1&p={}'''.format(group,pageNumber)
        p = opener.open(url)
        page = p.read()
        #print page
        response = BeautifulSoup( page , "html.parser", parse_only=SoupStrainer("div",{'class':'items_list_text'}) )

        for a in BeautifulSoup( str(response) , "html.parser", parse_only=SoupStrainer("a") ):
            if 'href' in a.attrs:
                if re.search("\/news\/[0-9]+", a['href']):
                    sports_pages.append(a['href'])


    sports_pages = set(sports_pages)                
    sport_texts = []
    print group
    from tqdm import tqdm
    for link in tqdm(set(sports_pages)): 
        sport_texts.append(getPageText(link))
    print group + ": " + str(len(sports_pages))
    return sport_texts

    
groups = ['15', # politics
          '24']  # science
         
group_texts = dict()

for g in groups:
    group_texts[g] = grabNewForGroup(g, 50)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [02:21<00:00,  3.43it/s]

15
15: 499
24


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [02:36<00:00,  2.36it/s]


24: 500


In [3]:
news_corpus = {}
news_corpus['politics'] = group_texts['15']
news_corpus['science'] = group_texts['24']

In [4]:
# save
import pickle
with open('dump.txt', 'w') as f:
    pickle.dump(news_corpus, f)

In [ ]:
# load
with open('dump.txt', 'r') as f:
    news_corpus = pickle.load(f)

In [6]:
print len(news_corpus['politics'])
print len(news_corpus['science'])

499
500


In [7]:
def load_ngrams(input_file):
    input_f = io.open(input_file, "r", encoding='utf8')
    n_dict = {}
    for line in input_f:
        line = line.replace('\n', '').split('\t')
        n_dict[tuple(line[0:])] = int(line[0])
    input_f.close()
    return n_dict

In [8]:
bigrams = load_ngrams('2grams-3.txt')
print '2grams count: ', len(bigrams.keys())

threegrams = load_ngrams('3grams-3.txt')
print '3grams count: ', len(threegrams.keys())

2grams count:  6750525
3grams count:  4655170


In [19]:
tokenized_corpus = defaultdict(list)
for key in news_corpus.keys():
    for text in news_corpus[key]:
        text_str = ''.join(text)
        line = nltk.word_tokenize(text_str)
        line = [word.encode("utf-8").decode("utf-8").lower() for word in line if word not in string.punctuation
                        and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]]
        tokenized_corpus[key] += line

In [20]:
for key in tokenized_corpus.keys():
    print(key, len(tokenized_corpus[key]))

('politics', 281647)
('science', 270218)


In [21]:
def find_perplexity(category, n3_gram, n2_gram):
    sum_probability = 0
    unknown_words = 0
    min_probability = 1
    sum_log = 0
    
    current_probability= {}
    
    for i in range(2, len(category)):
        try:
            current_probability[i] = (n3_gram[(category[i-2], category[i-1], category[i])] + 0.0) 
            / n2_gram[(category[i-2],category[i-1])]
            sum_probability += current_probability
            if current_probability[i] < min_probability:
                min_probability = current_probability[i]
        except KeyError:
            unknown_words +=1
    print (unknown_words, len(category), sum_probability)
    known_words = len(category) - unknown_words
    min_probability /= 10.0
    probability_to_substract = (unknown_words * min_probability + 0.0) / known_words
    for i in range(2, len(category)):
        try:
            sum_log += math.log(current_probability[i] - probability_to_substract, 2)
        except KeyError:
            sum_log += math.log(min_probability, 2)

    return sum_log

In [22]:
for key in tokenized_corpus.keys():
    perplexity = find_perplexity(tokenized_corpus[key], bigrams, threegrams)
    perplexity = 2**(-perplexity / len(tokenized_corpus[key]))
    print key,' perplexity: ', perplexity

(281645, 281647, 0)
politics  perplexity:  9.99983649264
(270216, 270218, 0)
science  perplexity:  9.99982957709
